## Построение линейной регрессионной модели

In [1]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from utils.preformatting_data import preformat
from sklearn.pipeline import Pipeline
import scikitplot as skplt
import pandas as pd
import numpy as np


data = preformat(pd.read_csv('../medics_1.csv', delimiter=',', encoding='utf-8'))

x = data[["возраст", "время_пережатия_аорты", "объем_кровопотерии",
          "есть_хроническое_заболевание", "переливание_крови_и_аик",
          "чсс", "мочевина", "длительность_аик"]]

y = data["калий"]

Нецелевые факторы для модели имеют воздействие на развитие ОПП, не имеют значимой корреляции

In [2]:
x_train, x_test, y_train, y_test = train_test_split(x, data["калий"],
                                                    test_size=0.13, random_state=42)

preprocessor = ColumnTransformer(
    transformers=[
        ('ill_category', OneHotEncoder(), ["есть_хроническое_заболевание"]),
        ('aik_blood_category', OneHotEncoder(), ['переливание_крови_и_аик'])
    ],
    remainder='passthrough'
)

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

Необходимые категориальные факторы были преобразованы с помощью one hot encoder

In [3]:
model.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ill_category',
                                                  OneHotEncoder(),
                                                  ['есть_хроническое_заболевание']),
                                                 ('aik_blood_category',
                                                  OneHotEncoder(),
                                                  ['переливание_крови_и_аик'])])),
                ('regressor', LinearRegression())])

In [4]:
y_pred = model.predict(x_test)

r2 = r2_score(y_test, y_pred)

mse = mean_squared_error(y_test, y_pred)

print("R-squared:", round(r2, 4))

print("Mean Squared Error:", round(mse, 2))

cv_scores = cross_val_score(model, x, data["калий"], cv=5, scoring='neg_mean_squared_error')

print("Cross-Validation Mean Squared Error:", round(-cv_scores.mean(), 2))

rounded_predictions = np.round(y_pred, 3).astype(float)

print(rounded_predictions)

print(f"Percentage mean absolute error: {round(mean_absolute_percentage_error(y_test, y_pred) * 100, 2)}%")

R-squared: 0.1217
Mean Squared Error: 0.16
Cross-Validation Mean Squared Error: 0.19
[4.39  4.431 4.35  4.523 4.412 4.492 4.392 4.551 4.396 4.372 4.554 4.488
 4.393 4.315 4.425 4.465 4.434 4.336 4.348 4.547 4.414]
Percentage mean absolute error: 7.99%


### Оценка модели регрессии

- **R-squared (коэффициент детерминации)**: 0.1217
- **Mean Squared Error (средняя квадратичная ошибка)**: 0.16
- **Cross-Validation Mean Squared Error (средняя квадратичная ошибка перекрестной проверки)**: 0.19
- **Прогнозы (округленные до 3 знаков после запятой)**:
  - [4.39, 4.431, 4.35, 4.523, 4.412, 4.492, 4.392, 4.551, 4.396, 4.372, 4.554, 4.488, 4.393, 4.315, 4.425, 4.465, 4.434, 4.336, 4.348, 4.547, 4.414]
- **Percentage mean absolute error (средняя абсолютная ошибка в процентах)**: 7.99%

#### Общий вывод:

R2 метрика, учитывая уникальность датасета, является приемлемой. Калий выбран в качестве целевого фактора, так как он имеет достаточную связь с развитием ОПП (маркер). Ошибки не являются наименьшими, однако это не исключает полезность модели.
